In [2]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm

def scale_mouse_position(x, y, original_width, original_height, new_width=128, new_height=128):
    scaled_x = (x / original_width) * new_width
    scaled_y = (y / original_height) * new_height
    return scaled_x, scaled_y

def get_next_clip_number(output_dir, base_filename):
    existing_clips = [f for f in os.listdir(output_dir) if f.startswith(base_filename) and f.endswith('.mp4')]
    return len(existing_clips) + 1

def process_and_save_videos(input_video_dir, input_json_dir, output_dir, sequence_length=128):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video_files = [f for f in os.listdir(input_video_dir) if f.endswith('.mp4')]
    
    for video_file in tqdm(video_files, desc="Processing videos"):
        video_path = os.path.join(input_video_dir, video_file)
        json_path = os.path.join(input_json_dir, os.path.splitext(video_file)[0] + '_input.json')
        
        # Open video file
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error opening video file {video_file}")
            continue

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Load JSON data
        try:
            with open(json_path, 'r') as f:
                json_data = json.load(f)
        except FileNotFoundError:
            print(f"JSON file not found for {video_file}")
            continue
        except json.JSONDecodeError:
            print(f"Error decoding JSON for {video_file}")
            continue

        base_filename = os.path.splitext(video_file)[0]
        clip_number = get_next_clip_number(output_dir, base_filename)

        # Process video in segments
        for start_frame in range(0, frame_count, sequence_length):
            end_frame = min(start_frame + sequence_length, frame_count)
            
            # Extract video segment
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            
            output_video_path = os.path.join(output_dir, f"{base_filename}_clip{clip_number:06d}.mp4")
            out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (128, 128))
            
            segment_frame_count = 0
            for _ in range(start_frame, end_frame):
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (128, 128))
                out.write(frame)
                segment_frame_count += 1
            
            out.release()
            
            # Check if the video segment was written successfully
            if segment_frame_count == 0:
                print(f"Error: No frames written for clip {clip_number} in {video_file}")
                continue

            # Extract and save corresponding JSON segment with scaled mouse positions
            json_segment = json_data[start_frame:start_frame+segment_frame_count]
            for frame_data in json_segment:
                x, y = frame_data['mouse_position']
                frame_data['mouse_position'] = list(scale_mouse_position(x, y, original_width, original_height))

            output_json_path = os.path.join(output_dir, f"{base_filename}_clip{clip_number:06d}_input.json")
            with open(output_json_path, 'w') as f:
                json.dump(json_segment, f)
            
            clip_number += 1
        
        cap.release()

# Usage
input_video_directory = 'Videos'
input_json_directory = 'Videos'
output_directory = 'output'

process_and_save_videos(input_video_directory, input_json_directory, output_directory)

print("Processing complete. Check the output directory for the segmented videos and JSON files.")

Processing videos: 100%|██████████| 1/1 [07:38<00:00, 458.21s/it]

Processing complete. Check the output directory for the segmented videos and JSON files.
